In [1]:
# use flirt to do linear transformation
import os
from nipype.interfaces.fsl import FLIRT
from nipype.interfaces.fsl import ApplyXFM
import nibabel as nib
import nilearn.datasets
from nibabel.affines import apply_affine
import glob
import numpy as np
import pandas as pd
from nilearn.image import resample_to_img


In [47]:
atlas = nilearn.datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-1mm')
atlas_img = nib.load(atlas['filename'])
atlas_data = atlas_img.get_fdata()

lobe_mapping = {
    'Frontal Pole': 'Frontal',
    'Insular Cortex': 'Insula',
    'Superior Frontal Gyrus': 'Frontal',
    'Middle Frontal Gyrus': 'Frontal',
    'Inferior Frontal Gyrus, pars opercularis': 'Frontal',
    'Inferior Frontal Gyrus, pars triangularis': 'Frontal',
    'Inferior Frontal Gyrus, pars orbitalis': 'Frontal',
    'Precentral Gyrus': 'Frontal',
    'Temporal Pole': 'Temporal',
    'Superior Temporal Gyrus, anterior division': 'Temporal',
    'Superior Temporal Gyrus, posterior division': 'Temporal',
    'Middle Temporal Gyrus, anterior division': 'Temporal',
    'Middle Temporal Gyrus, posterior division': 'Temporal',
    'Middle Temporal Gyrus, temporooccipital part': 'Temporal',
    'Inferior Temporal Gyrus, anterior division': 'Temporal',
    'Inferior Temporal Gyrus, posterior division': 'Temporal',
    'Inferior Temporal Gyrus, temporooccipital part': 'Temporal',
    'Postcentral Gyrus': 'Parietal',
    'Superior Parietal Lobule': 'Parietal',
    'Supramarginal Gyrus, anterior division': 'Parietal',
    'Supramarginal Gyrus, posterior division': 'Parietal',
    'Angular Gyrus': 'Parietal',
    'Lateral Occipital Cortex, superior division': 'Occipital',
    'Lateral Occipital Cortex, inferior division': 'Occipital',
    'Intracalcarine Cortex': 'Occipital',
    'Frontal Medial Cortex': 'Frontal',
    'Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)': 'Frontal',
    'Subcallosal Cortex': 'Frontal',
    'Paracingulate Gyrus': 'Frontal',
    'Cingulate Gyrus, anterior division': 'Frontal',
    'Cingulate Gyrus, posterior division': 'Parietal',
    'Precuneus Cortex': 'Parietal',
    'Cuneal Cortex': 'Occipital',
    'Frontal Orbital Cortex': 'Frontal',
    'Parahippocampal Gyrus, anterior division': 'Temporal',
    'Parahippocampal Gyrus, posterior division': 'Temporal',
    'Supracalcarine Cortex': 'Occipital',
    'Occipital Pole': 'Occipital',
    'Planum Temporale': 'Temporal',
    'Planum Polare': 'Temporal',
    'Heschl’s Gyrus (includes H1 and H2)': 'Temporal',
    'Temporal Fusiform Cortex, anterior division': 'Temporal',
    'Temporal Fusiform Cortex, posterior division': 'Temporal',
    'Temporal Occipital Fusiform Cortex': 'Temporal',
    'Occipital Fusiform Gyrus': 'Occipital',
    'Frontal Operculum Cortex': 'Frontal',
    'Central Operculum Cortex': 'Frontal',
    'Parietal Operculum Cortex': 'Parietal',
    'Lateral Occipital Cortex, superior division': 'Occipital',
    'Lateral Occipital Cortex, inferior division': 'Occipital',
    'Insular Cortex': 'Insula',
    'Frontal Opercular Cortex': 'Frontal',
    'Parietal Opercular Cortex': 'Parietal',
    'Central Opercular Cortex': 'Frontal',
    'Lingual Gyrus': 'Occipital',
    'Precuneous Cortex': 'Parietal',
}


[fetch_atlas_harvard_oxford] Dataset found in /home/vscode/nilearn_data/fsl

In [48]:
len(lobe_mapping)

53

In [49]:
def get_lobe_from_mni_coords(x, y, z, atlas_img, atlas_labels, lobe_mapping):
    atlas_data = atlas_img.get_fdata()
    # Map MNI to voxel index (affine is world-to-index)
    ijk = np.round(
        np.linalg.inv(atlas_img.affine).dot([x, y, z, 1])
    )[:3].astype(int)

    shape = atlas_data.shape
    if not all(0 <= idx < dim for idx, dim in zip(ijk, shape)):
        found_label = False
    else:
        label_val = int(atlas_data[tuple(ijk)])
        found_label = label_val != 0

    if found_label:
        region_name = atlas_labels[label_val - 1]
        lobe = lobe_mapping.get(region_name, None)
        return lobe, region_name, tuple(ijk)

    # If centroid voxel is background, find nearest labeled voxel
    labeled_voxels = np.array(np.where(atlas_data != 0)).T
    if labeled_voxels.size == 0:
        return None, None, None  # All atlas voxels are background

    # Compute distances in voxel space
    dists = np.sum((labeled_voxels - ijk)**2, axis=1)
    min_idx = np.argmin(dists)
    nearest_voxel = labeled_voxels[min_idx]
    label_val = int(atlas_data[tuple(nearest_voxel)])
    region_name = atlas_labels[label_val - 1]
    lobe = lobe_mapping.get(region_name, None)
    return lobe, region_name, tuple(nearest_voxel)


In [50]:
data_root = '/workspaces/data/brain_meningioma/bet'
cat = 'test'

mask_paths = sorted(glob.glob(f'{data_root}/b_{cat}_mni/*_gtv_mask.nii.gz*'))
list = []
for i, maskpath in enumerate(mask_paths):
    id = os.path.split(maskpath)[-1].split('-')[3]
    # if id!='0041':
    #     continue
    # if i> 9:
    #     break
    print(f'{i+1}: {maskpath}')
    mask_img = nib.load(maskpath)
    mask_data = mask_img.get_fdata() > 0.5
    coords = np.argwhere(mask_data > 0)
    centroid_voxel = coords.mean(axis=0)
    centroid_mni = apply_affine(mask_img.affine, centroid_voxel)
    x, y, z = centroid_mni
    print("centroid: ", x, y, z)
    
    # make_mni(maskpath)
    list.append([os.path.split(maskpath)[-1].split('-')[3], 
                 *get_lobe_from_mni_coords(x, y, z, atlas_img, atlas['labels'], lobe_mapping)])

1: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0011-1_t1c_gtv_mask.nii.gz
centroid:  12.841367221735325 -13.331288343558285 -36.303242769500436
2: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0020-1_t1c_gtv_mask.nii.gz
centroid:  -4.115241635687738 9.985130111524171 -25.245353159851298
3: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0022-1_t1c_gtv_mask.nii.gz
centroid:  -29.829046898638424 61.84266263237518 22.29954614220877
4: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0041-1_t1c_gtv_mask.nii.gz
centroid:  -23.501955207963036 -79.37575542125845 -23.61429079274796
5: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0045-1_t1c_gtv_mask.nii.gz
centroid:  13.070416095107447 -20.49748513946045 -27.406492912665755
6: /workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0053-1_t1c_gtv_mask.nii.gz
centroid:  -4.787258815701932 71.5705256154358 28.11194278110446
7: /workspaces/data/brain_meningioma/bet

In [51]:
dfRegion = pd.DataFrame(list,columns=['id', 'lobe', 'regionname', 'coords'])
dfRegion['lobe'].value_counts()

lobe
Frontal      29
Temporal      6
Parietal      6
Occipital     4
Insula        2
Name: count, dtype: int64

In [52]:
data_root = '/workspaces/data/brain_meningioma/bet'
cat = 'val'

mask_paths = sorted(glob.glob(f'{data_root}/b_{cat}_mni/*_gtv_mask.nii.gz*'))
listV = []
for i, maskpath in enumerate(mask_paths):
    id = os.path.split(maskpath)[-1].split('-')[3]
    # if id!='0041':
    #     continue
    # if i> 9:
    #     break
    print(f'{i+1}: {maskpath}')
    mask_img = nib.load(maskpath)
    mask_data = mask_img.get_fdata() > 0.5
    coords = np.argwhere(mask_data > 0)
    centroid_voxel = coords.mean(axis=0)
    centroid_mni = apply_affine(mask_img.affine, centroid_voxel)
    x, y, z = centroid_mni
    print("centroid: ", x, y, z)
    
    # make_mni(maskpath)
    listV.append([os.path.split(maskpath)[-1].split('-')[3], 
                 *get_lobe_from_mni_coords(x, y, z, atlas_img, atlas['labels'], lobe_mapping)])

    dfRegionV = pd.DataFrame(listV,columns=['id', 'lobe', 'regionname', 'coords'])

1: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0028-1_t1c_gtv_mask.nii.gz


centroid:  29.72801302931596 -38.739413680781766 -32.545602605863195
2: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0090-1_t1c_gtv_mask.nii.gz
centroid:  -3.549474319633788 38.317873132451325 -15.223451883897582
3: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0100-1_t1c_gtv_mask.nii.gz
centroid:  -5.669608189198726 -60.82809742322627 48.41016590187081
4: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0135-1_t1c_gtv_mask.nii.gz
centroid:  12.369440632819433 1.1625635644523413 -30.584147227379127
5: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0152-1_t1c_gtv_mask.nii.gz
centroid:  -8.358899723475474 -36.07247853296464 74.53558433997964
6: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0170-1_t1c_gtv_mask.nii.gz
centroid:  43.307134220072555 -69.89298669891173 -20.19528415961306
7: /workspaces/data/brain_meningioma/bet/b_val_mni/BraTS-MEN-RT-0172-1_t1c_gtv_mask.nii.gz
centroid:  62.698098256735335 -28.43383518225

In [53]:
dfRegion[dfRegion['lobe'].isna()]


,id,lobe,regionname,coords
2,0022,None,Background,"(119, 187, 94)"
5,0053,None,Background,"(97, 194, 97)"
22,0246,None,Background,"(117, 194, 73)"


In [54]:
dfRegionV[dfRegionV['lobe'].isna()]

,id,lobe,regionname,coords
10,0195,None,Background,"(103, 167, 111)"
16,0267,None,Background,"(118, 180, 96)"
32,0464,None,Background,"(82, 167, 123)"
39,0532,None,Background,"(83, 180, 53)"


In [55]:
dfRegionV['lobe'].value_counts()

lobe
Frontal      16
Temporal     13
Occipital     6
Parietal      4
Insula        2
Name: count, dtype: int64

In [ ]:
dfRA = pd.concat([dfRegion[['id','lobe']], dfRegionV[['id','lobe']] ],ignore_index=True)
dfRA['id'] = dfRA['id'].astype(str)
dfRA.to_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_lobe.csv',index=False)

In [123]:
dfRA.dtypes

id              object
lobe_overlap    object
dtype: object

In [ ]:
mask_img.shape

(197, 233, 189)

In [75]:
apply_affine(atlas_img.affine , np.argwhere(atlas_img.get_fdata()==1).mean(axis=0) )

array([ 2.60975791, 52.55349406,  7.86845587])

In [76]:
apply_affine(atlas_img.affine , np.argwhere(atlas_img.get_fdata()==48).mean(axis=0) )

array([ 4.45524504e-03, -9.58328926e+01,  7.50325209e+00])

'Frontal Pole'

In [42]:
np.argwhere(mask_img.get_fdata()>0).mean(axis=0)

array([ 93.21274118, 205.57052562, 100.11194278])

In [43]:
maskT = nib.load('/workspaces/data/brain_meningioma/bet/b_test_mni/BraTS-MEN-RT-0041-1_t1c_gtv_mask.nii.gz')

In [44]:
np.argwhere(maskT.get_fdata()>0).mean(axis=0)

array([74.49804479, 54.62424458, 48.38570921])

In [47]:
np.argwhere(mask_data>0).mean(axis=0)

array([ 93.21274118, 205.57052562, 100.11194278])

In [48]:
np.unique(atlas_data[maskT.get_fdata()>0])

array([ 0., 40.])

In [54]:
mask_img.affine

array([[   1.,    0.,    0.,  -98.],
       [   0.,    1.,    0., -134.],
       [   0.,    0.,    1.,  -72.],
       [   0.,    0.,    0.,    1.]])